In [1]:
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
import re

In [2]:
df=pd.read_csv('data/ContractNotice.csv')

C:\Users\mateu\AppData\Local\Temp\ipykernel_19392\1337289566.py:1: DtypeWarning: Columns (16) have mixed types. Specify dtype option on import or set low_memory=False.
  df=pd.read_csv('data/ContractNotice.csv')


In [3]:
def remove_none(cont):
    cont=eval(cont)
    return len([x for x in cont if not all(v is None for v in x.values())])

In [4]:
df['contractors']=df['contractors'].apply(remove_none)

In [4]:
len(df)

163183

In [3]:
ids=pd.read_csv('data_processed/ids_all.csv')

In [4]:
len(ids)

161508

In [7]:
df.drop(np.where(df['contractors']==0)[0], inplace=True)

In [8]:
df.drop_duplicates(inplace=True)

In [5]:
df=df[df['tenderId'].isin(ids['tenderId'])]

In [6]:
len(df)

161508

In [7]:
len(ids)

161508

In [10]:
df.columns

Index(['orderType', 'isTenderAmountBelowEU', 'orderObject', 'cpvCode',
       'tenderId', 'htmlBody', 'contractors', 'objectId'],
      dtype='object')

In [9]:

df.drop(columns=['clientType', 'tenderType', 'noticeType', 'noticeNumber','bzpNumber', 'publicationDate', 'organizationName', 'organizationCity', 'submittingOffersDate', 'procedureResult', 'organizationProvince', 'organizationCountry', 'organizationNationalId', 'organizationId'], inplace=True)

In [13]:
def remove_tags(html):
    soup=BeautifulSoup(html)
    txt=soup.get_text()
    c={}
    txt=list(txt.split())
    pattern=r"(\d+\.\d+\.?\))"
    num_pattern=r"(\d+\.\d+)"
    for x in range(len(txt)):
    
        if txt[x]=='PLN':
            i=x-1

            p=[]
            while True:
                i-=1
                if re.search(pattern, txt[i]) is not None:
                
                    p.append(txt[i])
                    break
            
            t=' '.join(p)
            cand=txt[x-1]
            if '.' in cand:
                
                cand=cand.replace('.','')
     
            if ',-' in cand:
                cand=cand.replace('-', '0')
      
            if ',' in cand:
                cand=cand.replace(',','.')
     
            cand=re.findall(num_pattern, cand)
            if len(cand)>0:
                cand=float(cand[0])
                c[t]=cand
            
           

    return c

In [15]:
def remove_tags2(html):
    soup=BeautifulSoup(html)
    txt=soup.get_text()
    c={}
    txt=list(txt.split())
    
    for x in range(len(txt)-22):
        if txt[x] in  '4.3.10.)':
            if txt[x+1]=='Zamawiający':
                c[txt[x]]= 0 if txt[x+22]=='Nie' else 1
        
        if txt[x] in  '7.5.)':
            if txt[x+1]=='Zamawiający':
                c[txt[x]]= 0 if txt[x+13]=='Nie' else 1
    return c

In [12]:
from copy import deepcopy

In [18]:
df.head()

,orderType,isTenderAmountBelowEU,orderObject,cpvCode,tenderId,htmlBody,contractors,objectId
0,Delivery,True,Dostawa gazów medycznych wraz z dzierżawą butl...,"24111500-0 (Gazy medyczne),24111000-5 (Wodór, ...",ocds-148610-65bc0cb3-86a2-11ed-94da-6ae0fe5e7159,"<html><head><meta charset=""UTF-8""><style>body ...",NaN,08daec88-a2be-9d2b-100a-0600107fa4a5
1,Delivery,True,1-P-23 Akcesoria do urządzeń medycznych,"33168000-5 (Przyrządy do endoskopii, endochiru...",ocds-148610-cd97182f-8a63-11ed-b4ea-f64d350121d2,"<html><head><meta charset=""UTF-8""><style>body ...",NaN,08daec89-e42d-79e6-e8e1-8400116e04c3
2,Delivery,True,Sukcesywna dostawa drobnego sprzętu laboratory...,"38000000-5 (Sprzęt laboratoryjny, optyczny i p...",ocds-148610-f8a2c4f0-8a65-11ed-b4ea-f64d350121d2,"<html><head><meta charset=""UTF-8""><style>body ...",NaN,08daec8b-e418-1fcc-100a-0600107fa4af
3,Delivery,True,2-P-23 Leki i inne produkty lecznicze,33600000-6 (Produkty farmaceutyczne),ocds-148610-3ee37822-8a68-11ed-94da-6ae0fe5e7159,"<html><head><meta charset=""UTF-8""><style>body ...",NaN,08daec8e-7fa7-7790-e8e1-8400116e04d8
4,Delivery,True,Sukcesywna dostawa podstawowych odczynników ch...,33696300-8 (Odczynniki chemiczne),ocds-148610-c61d386a-8a6f-11ed-94da-6ae0fe5e7159,"<html><head><meta charset=""UTF-8""><style>body ...",NaN,08daec95-45a7-1ee8-100a-0600107fa52d


In [30]:
a=df.iloc[0]['htmlBody']

In [ ]:
remove_tags()

{}

In [36]:
soup=BeautifulSoup(a)
txt=soup.get_text()
txt=list(txt.split())
txt[866]

'Nie'

In [44]:
subset=deepcopy(df[:10000])

In [16]:
df['htmlBody']=df['htmlBody'].apply(remove_tags2)

In [17]:
df.head()

,orderType,isTenderAmountBelowEU,orderObject,cpvCode,tenderId,htmlBody,contractors,objectId
1,Delivery,True,Dostawa gazów medycznych wraz z dzierżawą butl...,"24111500-0 (Gazy medyczne),24111000-5 (Wodór, ...",ocds-148610-65bc0cb3-86a2-11ed-94da-6ae0fe5e7159,"{'4.3.10.)': 0, '7.5.)': 0}",NaN,08daec88-a2be-9d2b-100a-0600107fa4a5
2,Delivery,True,1-P-23 Akcesoria do urządzeń medycznych,"33168000-5 (Przyrządy do endoskopii, endochiru...",ocds-148610-cd97182f-8a63-11ed-b4ea-f64d350121d2,"{'4.3.10.)': 0, '7.5.)': 0}",NaN,08daec89-e42d-79e6-e8e1-8400116e04c3
3,Delivery,True,Sukcesywna dostawa drobnego sprzętu laboratory...,"38000000-5 (Sprzęt laboratoryjny, optyczny i p...",ocds-148610-f8a2c4f0-8a65-11ed-b4ea-f64d350121d2,"{'4.3.10.)': 0, '7.5.)': 0}",NaN,08daec8b-e418-1fcc-100a-0600107fa4af
4,Delivery,True,2-P-23 Leki i inne produkty lecznicze,33600000-6 (Produkty farmaceutyczne),ocds-148610-3ee37822-8a68-11ed-94da-6ae0fe5e7159,"{'4.3.10.)': 0, '7.5.)': 0}",NaN,08daec8e-7fa7-7790-e8e1-8400116e04d8
5,Delivery,True,Sukcesywna dostawa podstawowych odczynników ch...,33696300-8 (Odczynniki chemiczne),ocds-148610-c61d386a-8a6f-11ed-94da-6ae0fe5e7159,"{'4.3.10.)': 0, '7.5.)': 0}",NaN,08daec95-45a7-1ee8-100a-0600107fa52d


In [18]:
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
import re
import json

In [20]:
df=pd.read_csv('./data_processed/full_contracts.csv')
df['htmlBody'] = df['htmlBody'].apply(lambda x: eval(x))

ParserError: Error tokenizing data. C error: Calling read(nbytes) on source failed. Try engine='python'.

In [ ]:
def one_hot_cpv(df):
    pattern = r'\d+-\d'
    matches = df['cpvCode'].apply(lambda x: re.findall(pattern, x))
    unique_matches = set([item for sublist in matches for item in sublist])
    for match in unique_matches:
        df[match] = df['cpvCode'].apply(lambda x: 1 if match in re.findall(pattern, x) else 0)

In [ ]:
one_hot_cpv(df)
df.head()

In [ ]:
unique_keys = set(key for d in df["htmlBody"] for key in d.keys())
expanded_df = pd.DataFrame(
    [{key: d.get(key, 0) for key in unique_keys} for d in df["htmlBody"]]
)
df = pd.concat([df, expanded_df], axis=1)

In [ ]:
df.drop(columns=["cpvCode", "htmlBody"], inplace=True)

In [ ]:
df.head()

In [ ]:
df.to_csv('data_processed/fullcontracts_oneHot.csv')

In [47]:
subset.to_csv('data_processed/subset10k.csv')

In [39]:
df=pd.read_csv('data/ContractNotice.csv')

C:\Users\creep\AppData\Local\Temp/ipykernel_24496/1337289566.py:1: DtypeWarning: Columns (16) have mixed types. Specify dtype option on import or set low_memory=False.
  df=pd.read_csv('data/ContractNotice.csv')


In [42]:
subset_df

,clientType,orderType,tenderType,noticeType,noticeNumber,bzpNumber,isTenderAmountBelowEU,publicationDate,orderObject,cpvCode,...,organizationName,organizationCity,organizationProvince,organizationCountry,organizationNationalId,organizationId,tenderId,htmlBody,contractors,objectId
1,1.1.12,Delivery,1.1.1,ContractNotice,2023/BZP 00000025/01,2023/BZP 00000025,True,2023-01-02T06:14:41.624Z,Dostawa gazów medycznych wraz z dzierżawą butl...,"24111500-0 (Gazy medyczne),24111000-5 (Wodór, ...",...,POWIATOWY ZAKŁAD OPIEKI ZDROWOTNEJ,Starachowice,PL26,PL,6641873185,7730,ocds-148610-65bc0cb3-86a2-11ed-94da-6ae0fe5e7159,"<html><head><meta charset=""UTF-8""><style>body ...",NaN,08daec88-a2be-9d2b-100a-0600107fa4a5
2,1.4,Delivery,1.1.1,ContractNotice,2023/BZP 00000028/01,2023/BZP 00000028,True,2023-01-02T06:23:40.899Z,1-P-23 Akcesoria do urządzeń medycznych,"33168000-5 (Przyrządy do endoskopii, endochiru...",...,Szpital Miejski nr 4 w Gliwicach Sp. z o.o.,Gliwice,PL24,PL,6312658474,10257,ocds-148610-cd97182f-8a63-11ed-b4ea-f64d350121d2,"<html><head><meta charset=""UTF-8""><style>body ...",NaN,08daec89-e42d-79e6-e8e1-8400116e04c3
3,1.4,Delivery,1.1.1,ContractNotice,2023/BZP 00000040/01,2023/BZP 00000040,True,2023-01-02T06:37:59.752Z,Sukcesywna dostawa drobnego sprzętu laboratory...,"38000000-5 (Sprzęt laboratoryjny, optyczny i p...",...,INSTYTUT ZOOTECHNIKI - PAŃSTWOWY INSTYTUT BADA...,Kraków,PL12,PL,6750002130,4411,ocds-148610-f8a2c4f0-8a65-11ed-b4ea-f64d350121d2,"<html><head><meta charset=""UTF-8""><style>body ...",NaN,08daec8b-e418-1fcc-100a-0600107fa4af
4,1.4,Delivery,1.1.1,ContractNotice,2023/BZP 00000069/01,2023/BZP 00000069,True,2023-01-02T06:56:39.735Z,2-P-23 Leki i inne produkty lecznicze,33600000-6 (Produkty farmaceutyczne),...,Szpital Miejski nr 4 w Gliwicach Sp. z o.o.,Gliwice,PL24,PL,6312658474,10257,ocds-148610-3ee37822-8a68-11ed-94da-6ae0fe5e7159,"<html><head><meta charset=""UTF-8""><style>body ...",NaN,08daec8e-7fa7-7790-e8e1-8400116e04d8
5,1.4,Delivery,1.1.1,ContractNotice,2023/BZP 00000264/01,2023/BZP 00000264,True,2023-01-02T07:45:08.9Z,Sukcesywna dostawa podstawowych odczynników ch...,33696300-8 (Odczynniki chemiczne),...,INSTYTUT ZOOTECHNIKI - PAŃSTWOWY INSTYTUT BADA...,Kraków,PL12,PL,6750002130,4411,ocds-148610-c61d386a-8a6f-11ed-94da-6ae0fe5e7159,"<html><head><meta charset=""UTF-8""><style>body ...",NaN,08daec95-45a7-1ee8-100a-0600107fa52d
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
232136,1.5,Delivery,1.1.1,ContractNotice,2024/BZP 00585732/01,2024/BZP 00585732,True,2024-11-08T08:41:59.1614016Z,236/2024 Nasienie buhaja rasy PHF odmiany czar...,03141000-1 (Nasienie byków),...,Instytut Zootechniki Państwowy Instytut Badawc...,Pawłowice,PL30,PL,6750002130,902,ocds-148610-4c0494e6-e50b-46af-8804-203b43aac5e4,"<html><head><meta charset=""UTF-8""><style>body ...",NaN,08dcffd1-3593-fa7d-985c-bb00013481e0
232139,1.5,Delivery,1.1.1,ContractNotice,2024/BZP 00585773/01,2024/BZP 00585773,True,2024-11-08T08:47:26.0758782Z,237/2024 Nasienie buhaja rasy PHF odmiany czar...,03141000-1 (Nasienie byków),...,Instytut Zootechniki Państwowy Instytut Badawc...,Pawłowice,PL30,PL,6750002130,902,ocds-148610-e4436a94-5985-48ca-aad9-ceb5d7e8452e,"<html><head><meta charset=""UTF-8""><style>body ...",NaN,08dcffd1-fe1e-bb5a-985c-bb000134821d
232145,1.5,Delivery,1.1.1,ContractNotice,2024/BZP 00585805/01,2024/BZP 00585805,True,2024-11-08T08:52:08.2551538Z,238/2024 Nasienie buhaja rasy PHF odmiany czar...,03141000-1 (Nasienie byków),...,Instytut Zootechniki Państwowy Instytut Badawc...,Pawłowice,PL30,PL,6750002130,902,ocds-148610-f386f86c-dbd6-4f2d-9f00-c7f0167d1ee0,"<html><head><meta charset=""UTF-8""><style>body ...",NaN,08dcffd2-a09e-83a3-985c-bb000134824a
232150,1.5,Delivery,1.1.1,ContractNotice,2024/BZP 00585838/01,2024/BZP 00585838,True,2024-11-08T08:57:17.7524176Z,239/2024 Nasienie buhaja rasy PHF odmiany czar...,03141000-1 (Nasienie byków),...,Instytut Zootechniki Państwowy Instytut Badawc...,Pawłowi

In [44]:
ids_all=subset_df['tenderId']

In [46]:
ids_all.to_csv('data_processed/ids_all.csv')

In [47]:
subset_df.to_csv('data_processed/full_contracts.csv')

In [17]:
subset.head()

,orderType,bzpNumber,isTenderAmountBelowEU,orderObject,cpvCode,htmlBody,contractors
0,Services,2023/BZP 00000008,True,„Przewóz dzieci przedszkolnych i uczniów szkół...,60100000-9 (Usługi w zakresie transportu drogo...,"{'4.3.)': 431252.33, '4.3.1)': 399307.71, '4.5...",2
1,Services,2023/BZP 00000010,True,Usługi restauracyjne na rzecz uczniów Szkoły P...,55523100-3 (Usługi w zakresie posiłków szkolny...,"{'6.2.)': 1210100.0, '6.3.)': 1383400.0, '6.4....",1
2,Services,2023/BZP 00000022,True,Zimowe utrzymanie obiektów drogowych: chodnikó...,90620000-9 (Usługi odśnieżania),"{'4.3.)': 438662.19, '8.2.)': 703677.89}",1
3,Works,2023/BZP 00000031,True,Remont 5 lokali mieszkalnych zasobu Gminy Miej...,"45453000-7 (Roboty remontowe i renowacyjne),45...","{'4.3.)': 7407407.41, '4.3.1)': 238562.68, '4....",5
4,Services,2023/BZP 00000037,True,Świadczenie usług utrzymania czystości.,"90919200-4 (Usługi sprzątania biur),90919100-3...","{'4.3.)': 229952.0, '4.5.5.)': 109584.0, '6.2....",1
